In [120]:
import gudhi
from methods import *
import random
import matplotlib.pyplot as plt

# 0 - Choose the protein structure you want to study

In [121]:
structure_id = "1nd1"  #The code can be different if it is a molecule or a protein
file_name = "DATA//" + structure_id + ".pdb"
random.seed(27112000)

### 1 - Read the .pdb file and plot the 3D structure

In [122]:
#structure = PDB_mol_reader(structure_id, file_name)
structure = PDB_prot_reader(structure_id, file_name)
plot_3d(structure_id, structure)
print("Total number of carbon atoms: ", np.shape(structure[1])[0])

Total number of carbon atoms:  1596


### 2 - Analyse the 3D structure to determine the different values of epsilon

In [123]:
min = [np.min(idx) for idx in zip(*structure[1])]
max = [np.max(idx) for idx in zip(*structure[1])]

random_pair = [[random.randint(0, np.shape(structure[1])[0]-1), random.randint(0, np.shape(structure[1])[0]-1)]  for i in range(1000)]
dist = np.sort(np.array([np.linalg.norm(structure[1][idx[0]] - structure[1][idx[1]]) for idx in random_pair]))

while dist[0] == 0:
    dist = dist[1:]

min_dist, max_dist = np.min(dist), np.max(dist)

epsilon = np.linspace(min_dist*1.05, min_dist*1.4, 3)

print(" Min (x,y,z): ", min,'\n',
    "Max (x,y,z): ", max, '\n',
    "Min euclidean distance: ", min_dist, '\n',
    "Max euclidean distance: ", max_dist, '\n',
    "Epsilon range: ", epsilon, '\n'
    "Distance batch: ", dist[:10])

 Min (x,y,z):  [-16.322, 3.969, -2.615] 
 Max (x,y,z):  [28.38, 49.126, 43.564] 
 Min euclidean distance:  2.2474825 
 Max euclidean distance:  46.24859 
 Epsilon range:  [2.35985667 2.75316611 3.14647555] 
Distance batch:  [2.2474825 4.1645956 5.917309  6.113694  6.3181596 7.5526123 8.562862
 9.040073  9.561576  9.667909 ]


In [124]:
rips_complex = []
simplex_tree = []
fmt = '%s -> %.2f'
fig = plt.figure(figsize=(14, 30))
subplot = [311, 312, 313]

for i in range(3):
    rips_complex.append(gudhi.RipsComplex(points = structure[1], max_edge_length = epsilon[i]))
    simplex_tree.append(rips_complex[i].create_simplex_tree(max_dimension=2))

    print('Rips complex is of dimension '  + \
        repr(simplex_tree[i].dimension()) + ' - ' + \
        repr(simplex_tree[i].num_simplices()) + ' simplices - ' + \
        repr(simplex_tree[i].num_vertices()) + ' vertices.') 

    res = ""
    for filtered_value in simplex_tree[0].get_filtration():
        res += fmt % tuple(filtered_value)
    #print(res)

    triangles = np.array([s[0] for s in simplex_tree[i].get_skeleton(2) if len(s[0])==3])

    ax = fig.add_subplot(subplot[i], projection='3d')
    ax.set_title("Vietoris Rips complex with epsilon of " + str(round(epsilon[i], 2)))
    ax.scatter(structure[1][:, 0], structure[1][:, 1], structure[1][:, 2], s=1)
    if len(triangles) != 0:
        ax.plot_trisurf(structure[1][:, 0], structure[1][:, 1], structure[1][:, 2], triangles=triangles, alpha = 0.1)

    plot = pv.Plotter(off_screen=True)
    plot.add_mesh(structure[1], color = 'k', render_points_as_spheres=True, point_size=2, show_scalar_bar=True)
    for triangle in triangles:
        a = np.array([structure[1][triangle[0]][0], structure[1][triangle[0]][1], structure[1][triangle[0]][2]])
        b = np.array([structure[1][triangle[1]][0], structure[1][triangle[1]][1], structure[1][triangle[1]][2]])
        c = np.array([structure[1][triangle[2]][0], structure[1][triangle[2]][1], structure[1][triangle[2]][2]])
        tri = pv.Triangle([a, b, c])
        plot.add_mesh(tri, color = 'b', style='surface', show_edges=True, edge_color = 'k', opacity = 0.2, line_width=1)    
    plot.background_color = 'w'
    path = plot.generate_orbital_path(n_points=100, shift = 100, factor=3.0)
    plot.open_gif("RESULTS//Rips_" + str(round(epsilon[i], 2)) + "_" + structure_id + ".gif")
    plot.orbit_on_path(path, write_frames=True)
    plt.close()

plt.show()

Rips complex is of dimension 2 - 4090 simplices - 1596 vertices.
Rips complex is of dimension 2 - 8749 simplices - 1596 vertices.
Rips complex is of dimension 2 - 12539 simplices - 1596 vertices.
